In [10]:
# Import our dependencies
import tensorflow as tf
import pandas as pd
import tensorflow_hub as hub
import os
import re
from keras import backend as K
import keras.layers as layers
from keras.models import Model, load_model
from keras.engine import Layer
import numpy as np
from keras.datasets import imdb
from keras.preprocessing import sequence

# Initialize session
#sess = tf.Session()
#K.set_session(sess)

In [6]:
# Load all files from a directory in a DataFrame.
#def load_directory_data(directory):
#  data = {}
#  data["sentence"] = []
#  data["sentiment"] = []
#  for file_path in os.listdir(directory):
#    with tf.gfile.GFile(os.path.join(directory, file_path), "r") as f:
#      data["sentence"].append(f.read())
#      data["sentiment"].append(re.match("\d+_(\d+)\.txt", file_path).group(1))
#  return pd.DataFrame.from_dict(data)
#
## Merge positive and negative examples, add a polarity column and shuffle.
#def load_dataset(directory):
#  pos_df = load_directory_data(os.path.join(directory, "pos"))
#  neg_df = load_directory_data(os.path.join(directory, "neg"))
#  pos_df["polarity"] = 1
#  neg_df["polarity"] = 0
#  return pd.concat([pos_df, neg_df]).sample(frac=1).reset_index(drop=True)
#
## Download and process the dataset files.
#def download_and_load_datasets(force_download=False):
#  dataset = tf.keras.utils.get_file(
#      fname="aclImdb.tar.gz", 
#      origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", 
#      extract=True)
#
#  train_df = load_dataset(os.path.join(os.path.dirname(dataset), 
#                                       "aclImdb", "train"))
#  test_df = load_dataset(os.path.join(os.path.dirname(dataset), 
#                                      "aclImdb", "test"))
#
#  return train_df, test_df
#
## Reduce logging output.
##tf.logging.set_verbosity(tf.logging.ERROR)
#
#train_df, test_df = download_and_load_datasets()
#train_df.head()

vocabulary_size = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words = vocabulary_size)
print('Loaded dataset with {} training samples, {} test samples'
      .format(len(X_train), len(X_test)))

Loaded dataset with 25000 training samples, 25000 test samples


In [7]:
word2id = imdb.get_word_index()
id2word = {i: word for word, i in word2id.items()}

In [11]:
max_words = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_test = sequence.pad_sequences(X_test, maxlen=max_words)

In [8]:
# Create a custom layer that allows us to update weights (lambda layers do not have trainable parameters!)

class ElmoEmbeddingLayer(Layer):
    def __init__(self, **kwargs):
        self.dimensions = 1024
        self.trainable=True
        super(ElmoEmbeddingLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.elmo = hub.Module('https://tfhub.dev/google/elmo/2', trainable=self.trainable,
                               name="{}_module".format(self.name))

        self.trainable_weights += K.tf.trainable_variables(scope="^{}_module/.*".format(self.name))
        super(ElmoEmbeddingLayer, self).build(input_shape)

    def call(self, x, mask=None):
        result = self.elmo(K.squeeze(K.cast(x, tf.string), axis=1),
                      as_dict=True,
                      signature='default',
                      )['default']
        return result

    def compute_mask(self, inputs, mask=None):
        return K.not_equal(inputs, '--PAD--')

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.dimensions)

In [ ]:
# Function to build model
#def build_model(): 
#    input_text = layers.Input(shape=(1,), dtype="string")
#    embedding = ElmoEmbeddingLayer()(input_text)
#    dense = layers.Dense(256, activation='relu')(embedding)
#    pred = layers.Dense(1, activation='sigmoid')(dense)
#
#    model = Model(inputs=[input_text], outputs=pred)
#
#    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#    model.summary()
#
#    return model

input_text = layers.Input(shape=(1,), dtype="string")
embedding = ElmoEmbeddingLayer()(input_text)
dense = layers.Dense(256, activation='relu')(embedding)
pred = layers.Dense(1, activation='sigmoid')(dense)

model = Model(inputs=[input_text], outputs=pred)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

In [7]:
# Create datasets (Only take up to 150 words for memory)
#train_text = train_df['sentence'].tolist()
#train_text = [' '.join(t.split()[0:150]) for t in train_text]
#train_text = np.array(train_text, dtype=object)[:, np.newaxis]
#train_label = train_df['polarity'].tolist()
#
#test_text = test_df['sentence'].tolist()
#test_text = [' '.join(t.split()[0:150]) for t in test_text]
#test_text = np.array(test_text, dtype=object)[:, np.newaxis]
#test_label = test_df['polarity'].tolist()

NameError: name 'train_df' is not defined

In [ ]:
# Build and fit
#model = build_model()
#model.fit(train_text, 
#          train_label,
#          validation_data=(test_text, test_label),
#          epochs=1,
#          batch_size=32)
# save best parameter estimate using val loss.
checkpointer = ModelCheckpoint("best_base_model.tf", monitor="val_loss", mode="auto", save_best_only = True, verbose=1)
# early stopping.
earlystoper = EarlyStopping(monitor = 'val_loss', patience = 2, verbose = 1) # patience is num. epochs wo improvement.

# save the training history.
csv_logger = CSVLogger('trainingBaseModel.log', separator=',', append=False) 
# compile the callbacks together
callbackser = [earlystoper, checkpointer, csv_logger]

batch_size = 64
num_epochs = 3
X_valid, y_valid = X_train[:batch_size], y_train[:batch_size]
X_train2, y_train2 = X_train[batch_size:], y_train[batch_size:]
model.fit(X_train2, y_train2, validation_data=(X_valid, y_valid), 
          batch_size=batch_size, epochs=num_epochs)

In [9]:
model.save('ElmoModel.h5')
pre_save_preds = model.predict(test_text[0:100]) # predictions before we clear and reload model

# Clear and load model
model = None
model = build_model()
model.load_weights('ElmoModel.h5')

post_save_preds = model.predict(test_text[0:100]) # predictions after we clear and reload model
all(pre_save_preds == post_save_preds) # Are they the same?

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 1)                 0         
_________________________________________________________________
elmo_embedding_layer_2 (Elmo (None, 1024)              4         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               262400    
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 257       
Total params: 262,661
Trainable params: 262,661
Non-trainable params: 0
_________________________________________________________________


True